## **Validación Cruzada**

En este Notebook, se realiza validación cruzada K-Fold al modelo entrenado sólo con CNN para la clasificación de acentos, con la finalidad de validar la estabilidad del modelo.

## Preprocesamiento

Se cargan los datos asociados a los espectrogramas de Mel.

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
# Cargar el archivo .npz con los datos
data = np.load("mel_Train.npz")

# Extraer características y etiquetas
X = data['X']  # features
y = data['y']  # Etiquetas de clase

In [ ]:
# obtener las clases únicas ordenadas (10 posibles combinaciones)
clases = sorted(set(y))

# crear un diccionario para mapear cada etiqueta a un número y viceversa
label_to_index = {}
index_to_label = {}
for i, etiqueta in enumerate(clases):
    label_to_index[etiqueta] = i  # '0 2' → 0, '0 3' → 1, etc.
    index_to_label[i] = etiqueta  #  0 → '0 2'

#transforma las etiquetas del conjunto y en números
y_ok = []
for etiqueta in y:
    indice = label_to_index[etiqueta]
    y_ok.append(indice)

y_ok = np.array(y_ok)


# codificar para clasificación, convertir a one-hot
y_final = to_categorical(y_ok, num_classes=len(clases))

print("Forma de X:", X.shape)
print("Forma de y_final:", y_final.shape)

Forma de X: (1395, 200, 128, 1)
Forma de y_final: (1395, 10)


In [ ]:
print("Etiqueta original:", y[0])
print("Índice asignado:", y_ok[0])
print("Vector one-hot:", y_final[0])
print("Decodificada:", index_to_label[np.argmax(y_final[0])])

Etiqueta original: 0 2
Índice asignado: 0
Vector one-hot: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Decodificada: 0 2


## Validación Cruzada

Para realizar la validación se utilizan 5 particiones de los datos, que luego son divididos en entrenamiento y validación

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score
import tensorflow as tf

#Se utiliza K-Fold, 5 grupos de data

y_labels = np.argmax(y_final, axis=1)  
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

f1_scores = []
recall_scores = []
val_losses= []
val_accuracies= []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y_labels)):
    print(f"\nFold {fold+1}")

    # Dividir los datos
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_final[train_idx], y_final[val_idx]

    # Cargar el modelo guardado anteriormente
    model = tf.keras.models.load_model("modelo_cnn_optuna.keras")

    # Entrenamiento con validación
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=70,
        batch_size=32,
        verbose=0,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
    )

    # Últimos valores de val_loss y val_accuracy para cada grupo
    val_loss = history.history['val_loss'][-1]
    val_acc = history.history['val_accuracy'][-1]
    
    #Se guardan y muestran los valores 
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    print(f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}")

    # Predecir con conjunto de validación
    y_pred = tf.argmax(model.predict(X_val), axis=1).numpy()
    y_true = tf.argmax(y_val, axis=1).numpy()

    # Calcular F1 macro Score para evaluar
    f1 = f1_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    #Guardo el F1 macro Score y Recall
    
    f1_scores.append(f1)
    recall_scores.append(recall)
    print(f"Fold {fold+1} - F1 macro: {f1:.4f} | Recall macro: {recall:.4f}")

# Promedio de las métricas obtenidas para cada fold
print(f"\nF1 macro promedio: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"\nRecall macro promedio: {np.mean(recall_scores):.4f} ± {np.std(recall_scores):.4f}")



Fold 1


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Val Loss: 1.0071 | Val Accuracy: 0.9104
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Fold 1 - F1 macro: 0.9217 | Recall macro: 0.9224

Fold 2


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Val Loss: 0.9677 | Val Accuracy: 0.9247
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Fold 2 - F1 macro: 0.9233 | Recall macro: 0.9249

Fold 3


...

Val Loss: 0.9975 | Val Accuracy: 0.9247
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Fold 3 - F1 macro: 0.9041 | Recall macro: 0.9015

Fold 4


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Val Loss: 0.9298 | Val Accuracy: 0.9104
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Fold 4 - F1 macro: 0.9417 | Recall macro: 0.9420

Fold 5


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Val Loss: 1.0708 | Val Accuracy: 0.8925
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Fold 5 - F1 macro: 0.9307 | Recall macro: 0.9318

F1 macro promedio: 0.9243 ± 0.0123

Recall macro promedio: 0.9245 ± 0.0134


## Validación

Los promedios se observan altos y estables.
F1 macro promedio: 0.9243 ± 0.0123
Recall macro promedio: 0.9245 ± 0.0134

Usamos validación cruzada con 5 folds para asegurarnos de que el rendimiento del modelo no dependa del azar en la separación de datos. Esta técnica nos permitió evaluar la robustez y generalización del sistema en distintas configuraciones, obteniendo un macro F1-score promedio alto y estable. Es especialmente útil ´porque trabajamos con una cantidad moderada de datos y queremos tener confianza en que el modelo no está sobreajustado a un subconjunto específico.